## Baselin model (KNN)

In [1]:
# importing useful libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_curve, auc, confusion_matrix, accuracy_score, precision_score, recall_score

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# loading in the dataset
df = pd.read_csv('creditcard.csv')
print(df.shape)
df.head()

(284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
# 27 most important features according to our EDA
cols = ['V'+str(i) for i in range(1, 29) if i != 25]
print(cols)

['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V26', 'V27', 'V28']


In [4]:
# selecting the 27 most important features according to our EDA 
X = df[cols]

y = df['Class'] # selecting the target variable

X.shape

(284807, 27)

In [5]:
val_count = df['Class'].value_counts()
weights = dict(1 / val_count) # to be used as class weights
weights

{0: 3.51722561243691e-06, 1: 0.0020325203252032522}

## <center>Cost matrix for fraud detection</center>

||Actual Positive|Actual Negative|
|:-:|:-:|:-:|
|**Predicted Positive**|administrative cost|administrative cost|
|**Predicted Negative**|transactional cost|0|

### *see [this article](https://towardsdatascience.com/fraud-detection-with-cost-sensitive-machine-learning-24b8760d35d9) for more about cost sensitive machine learning for fraud detection*

In [6]:
admin_cost = 2.5

### our choice of using 2.5euros as the administartive cost because it seems to be the best for our problem as suggested by [this paper](https://www.researchgate.net/publication/262390835_Cost_Sensitive_Credit_Card_Fraud_Detection_Using_Bayes_Minimum_Risk)

## Also, implementing cost sensitive models for sklearn's classifier is challenging. It is as good as creating a new model from the scratch. For simplicity, we will not penalize our model of True Positives (although, the cost matrix suggests we should penalize it for true positives). This will mean that we are only penalizing MISCLASSIFICATIONS (trying to minimize costs due to misclassification and not cost in general).

In [7]:
# defining a function to calculate cost savings
def cost_saving(ytrue, ypred, amount):
    fp = np.sum((ytrue == 0) & (ypred == 1))
    cost = np.sum(fp*admin_cost) + np.sum((amount[(ytrue == 1) & (ypred == 0)]))
    max_cost = np.sum((amount[(ytrue == 1)]))
    savings = 1 - (cost/max_cost)
    
    return savings

In [8]:
cv = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)

In [9]:
# defining a function to calculate cost saving per fold (splits) of our cv
def cost_saving_per_split(scores, x, y, cv_object):
    results = []
    for i, (_, test_ind) in zip(range(cv_object.n_splits), cv_object.split(x, y)):
        ypred = scores['estimator'][i].predict(x[test_ind])
        ytrue = y[test_ind]
        amount = df['Amount'].values[test_ind]
        results.append(cost_saving(ytrue, ypred, amount))
        
    return results

In [10]:
# defining a function to return a dataframe of metrics results for each fold in our cv
def get_metric_scores(scores, x, y=y, cv_object=cv):
    ind = ['split_'+str(n) for n in range(1, cv_object.n_splits+1)]
    
    scores_df = pd.DataFrame(index=ind)
    
    scores_df['f1_score'] = scores['test_f1']
    scores_df['auc_pr'] = scores['test_average_precision']
    scores_df['cost_savings'] = cost_saving_per_split(scores, x, y, cv_object)

    return scores_df

In [11]:
from sklearn.pipeline import Pipeline

## model 

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=0, stratify=df['Class'])

In [14]:
#knn
from termcolor import colored as cl

In [24]:
n = 5
knn = KNeighborsClassifier(n_neighbors = n)
knn.fit(x_train, y_train)
knn_yhat = knn.predict(x_test)

In [25]:
print(cl('F1 score of the KNN model is {}'.format(f1_score(y_test, knn_yhat)), attrs = ['bold'], color = 'green'))

F1 score of the KNN model is 0.81651376146789


In [26]:
def plot_confusion_matrix(cm, classes, title, normalize = False, cmap = plt.cm.Blues):
    title = 'Confusion Matrix of {}'.format(title)
    if normalize:
        cm = cm.astype(float) / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment = 'center',
                 color = 'white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [27]:
knn_matrix = confusion_matrix(y_test, knn_yhat, labels = [0, 1])
knn_matrix

array([[71073,     6],
       [   34,    89]], dtype=int64)

In [ ]:
#Model two

In [ ]:
myList = list(range(0,50))
neighbors = list(filter(lambda x: x%2!=0, myList))  #This will give a list of odd numbers only ranging from 0 to 50

CV_Scores = []

for k in neighbors:
    KNN = KNeighborsClassifier(n_neighbors = k, algorithm = 'kd_tree')
    scores = cross_val_score(KNN, x_train, y_train, cv = 4, scoring='recall')
    CV_Scores.append(scores.mean())

In [ ]:
CV_Scores

In [ ]:
best_k = neighbors[CV_Scores.index(max(CV_Scores))]
best_k

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
KNN_best = KNeighborsClassifier(n_neighbors = best_k, algorithm = 'kd_tree')

KNN_best.fit(x_train, y_train)

prediction = KNN_best.predict(x_test)

recallTest = recall_score(y_test, prediction)

F1test = f1_score(y_test, prediction)

print("F1 score Score of the knn classifier for best k values of "+str(best_k)+" is: "+str(F1test))
      
print("Recall Score of the knn classifier for best k values of "+str(best_k)+" is: "+str(recallTest))

#recall = recall_score(y_true, y_pred, average='binary')
#print('Recall: %.3f' % recall)

cm = confusion_matrix(y_test, prediction)

print(cm)

tn, fp, fn, tp = cm.ravel()

(tn, fp, fn, tp)

## Fraud Sensitive model (Not cost sensitive)

In [16]:
fraud_sensitive_model = KNeighborsClassifier()
n = 5

In [18]:
fraud_sensitive_model =  KNeighborsClassifier(n_neighbors = n, algorithm = 'kd_tree')
fraud_sensitive_scaler = StandardScaler()
fraud_sensitive_pipe = Pipeline([('scaler', fraud_sensitive_scaler), ('model', fraud_sensitive_model)])
fraud_sensitive_scores = cross_validate(fraud_sensitive_pipe, np.array(X), y, \
                            scoring=['f1', 'average_precision'], cv=cv, n_jobs=4, \
                                        return_estimator=True, error_score='raise')

### Note: we used cross_validate because it can take more than one scoring metrics and it can also return the fitting model for each fold.

In [19]:
fraud_sensitive_results = get_metric_scores(fraud_sensitive_scores, np.array(X))
fraud_sensitive_results

,f1_score,auc_pr,cost_savings
split_1,0.805310,0.757136,0.719787
split_2,0.874459,0.844667,0.725883
split_3,0.855856,0.856481,0.777775
split_4,0.830357,0.776869,0.695133


In [20]:
fraud_sensitive_results.mean()

f1_score        0.841495
auc_pr          0.808788
cost_savings    0.729645
dtype: float64

## Cost sensitive model

### Cant implent a cost sensitive model.... cant put our sample_weights in my model.

In [21]:
sample_weights = np.array([df['Amount'][ind] if fraud else admin_cost for ind, fraud in enumerate(y)])

## Bayes Mininmum Risk (BMR)

### Note: cost dependent classification is also called Bayes Mininmum Risk.
***see more about BMR [here](https://link.springer.com/article/10.1007/s42452-020-03375-w)***

In [28]:
scaler2 = StandardScaler()

bmr_model = KNeighborsClassifier(n_neighbors = n, algorithm = 'kd_tree')
bmr_pipe = Pipeline([('scaler', scaler2), ('model', bmr_model)])
bmr_scores = cross_validate(bmr_pipe, np.array(X), y, cv=cv, n_jobs=4, return_estimator=True, \
                            error_score='raise')

In [29]:
# defining a function to predict based on the predicting that will minimize the expected cost.
def bmr_predict(model, x, trans_cost):
    prob = model.predict_proba(x)[:, 1]
        
    expected_cost_0 = prob * trans_cost
    expected_cost_1 = (1-prob) * admin_cost
        
    pred = (expected_cost_1 < expected_cost_0).astype(int)
    return pred

In [30]:
def get_bmr_metric_scores(scores, x, y=y, cv_object=cv):
    ind = ['split_'+str(n) for n in range(1, cv_object.n_splits+1)]
    scores_df = pd.DataFrame(index=ind)

    f1_results = []
    cs_results = []
    
    for i, (_, test_ind) in zip(range(cv_object.n_splits), cv_object.split(x, y)):
        amount = df['Amount'].values[test_ind]
        
        ypred = bmr_predict(scores['estimator'][i], x[test_ind], amount)
        ytrue = y[test_ind]
                
        f1_results.append(f1_score(ytrue, ypred))
        cs_results.append(cost_saving(ytrue, ypred, amount))
        
    scores_df['f1_score'] = f1_results
    #scores_df['auc_pr'] = scores['test_average_precision']
    scores_df['cost_savings'] = cs_results

    return scores_df

In [31]:
bmr_results = get_bmr_metric_scores(bmr_scores, np.array(X))
bmr_results

,f1_score,cost_savings
split_1,0.679537,0.722176
split_2,0.688889,0.749834
split_3,0.720307,0.836207
split_4,0.682540,0.731263


In [32]:
bmr_results.mean()

f1_score        0.692818
cost_savings    0.759870
dtype: float64

In [33]:
from sklearn.metrics import roc_auc_score

In [34]:
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score

smote = SMOTE()

xx =np.array(X)

for train_idx, test_idx, in cv.split(xx, y):
    X_train, y_train = xx[train_idx], y[train_idx]
    X_test, y_test = xx[test_idx], y[test_idx]
    X_train_oversampled, y_train_oversampled = smote.fit_sample(X_train, y_train)
    smote_gradient_model = KNeighborsClassifier()
    smote_gradient_model.fit(X_train_oversampled, y_train_oversampled )  
    pred = smote_gradient_model.predict(X_test)
    print(f'roc_auc_score: {roc_auc_score(y_test, pred)}')
    print(f'f-score: {f1_score(y_test, pred)}')

roc_auc_score: 0.9179183656522337
f-score: 0.6112759643916914
roc_auc_score: 0.9341644594008933
f-score: 0.6239067055393587
roc_auc_score: 0.9423578505400553
f-score: 0.6488095238095237
roc_auc_score: 0.9096475828569874
f-score: 0.5690140845070423


In [35]:
print(f'roc_auc_score: {roc_auc_score(y_test, pred)}')
print(f'f-score: {f1_score(y_test, pred)}')

roc_auc_score: 0.9096475828569874
f-score: 0.5690140845070423


## We then see that for KNN,  the fraud sensitivite seems to be the best approach.